In [7]:
from cache import Cache
cache=Cache()

df = cache.load('dataframe_with_datetime')

# preprocessing data

In [11]:
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

from gensim.models import TfidfModel
from gensim.similarities import MatrixSimilarity

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

import pandas as pd

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize


import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk.internals
from gensim import corpora, models


from numpy import dot
from numpy.linalg import norm

from collections import defaultdict

In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

def tokenization(text):
    text = re.split('\W+', text)
    return text

stopword = nltk.corpus.stopwords.words('french')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

stemmer = FrenchStemmer()

def stemming(text):
    text = [stemmer.stem(word) for word in text]
    return text

wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

def tokenization(text):
    text = re.split('\W+', text)
    return text

stopword = nltk.corpus.stopwords.words('french')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

stemmer = FrenchStemmer()

def stemming(text):
    text = [stemmer.stem(word) for word in text]
    return text

wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

In [ ]:
df['content'] = df['content'].apply(lambda x: remove_punct(x))
#df['content'] = df['content'].apply(lambda x: tokenization(x.lower()))
df['content'] = df['content'].apply(lambda x: remove_stopwords(x))
df['content'] = df['content'].apply(lambda x: stemming(x))
df['content'] = df['content'].apply(lambda x: lemmatizer(x))

In [ ]:
corpus = df["content"].values
dictionary = corpora.Dictionary([d.split() for d in corpus])
corpus_gensim = [dictionary.doc2bow(doc.split()) for doc in corpus]   
tfidf = TfidfModel(corpus_gensim)
corpus_tfidf = tfidf[corpus_gensim]
#lsi = LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)

#On cherche ici le nomre optimal de topic entre les nombres: start et stop
coherence_values = []
model_list = []
start,stop,step=2,10,1
for num_topics in range(start, stop, step):
    # generate LSA model
    print(num_topics)
    model_Lsi = LsiModel(corpus_tfidf, num_topics=num_topics, id2word = dictionary)  # train model
    #model_list.append(model_Lsi) Pour pas refaire le meilleur modèle
    coherencemodel = CoherenceModel(model=model_Lsi, texts=[d.split() for d in corpus], dictionary=dictionary, coherence='c_v')
    coherence_values.append(coherencemodel.get_coherence())

num_topics_optimal = coherence_values.index(max(coherence_values))+ start

print("nombre optimal de topics:"+ num_topics_optimal)
model_Lsi = LsiModel(corpus_tfidf, num_topics=num_topics_optimal, id2word = dictionary)  # train model

def text2Vec_LSA(text):
    vec = dictionary.doc2bow(text.lower().split())   
    vec_lsi = lsi[vec]
    return(vec_lsi)

df['Vec_LSA'] = df['content'].apply(lambda x: text2Vec_LSA(x))

In [28]:
string_list = [
    "corpus = df[\"content\"].values\n",
    "dictionary = corpora.Dictionary([d.split() for d in corpus])\n",
    "corpus_gensim = [dictionary.doc2bow(doc.split()) for doc in corpus]   \n",
    "tfidf = TfidfModel(corpus_gensim)\n",
    "corpus_tfidf = tfidf[corpus_gensim]\n",
    "#lsi = LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)\n",
    "\n",
    "#On cherche ici le nomre optimal de topic entre les nombres: start et stop\n",
    "coherence_values = []\n",
    "model_list = []\n",
    "start,stop,step=2,10,1\n",
    "for num_topics in range(start, stop, step):\n",
    "    # generate LSA model\n",
    "    print(num_topics)\n",
    "    model_Lsi = LsiModel(corpus_tfidf, num_topics=num_topics, id2word = dictionary)  # train model\n",
    "    #model_list.append(model_Lsi) Pour pas refaire le meilleur modèle\n",
    "    coherencemodel = CoherenceModel(model=model_Lsi, texts=[d.split() for d in corpus], dictionary=dictionary, coherence='c_v')\n",
    "    coherence_values.append(coherencemodel.get_coherence())\n",
    "\n",
    "num_topics_optimal = coherence_values.index(max(coherence_values))+ start\n",
    "\n",
    "print(\"nombre optimal de topics:\"+ num_topics_optimal)\n",
    "model_Lsi = LsiModel(corpus_tfidf, num_topics=num_topics_optimal, id2word = dictionary)  # train model\n",
    "\n",
    "def text2Vec_LSA(text):\n",
    "    vec = dictionary.doc2bow(text.lower().split())   \n",
    "    vec_lsi = lsi[vec]\n",
    "    return(vec_lsi)\n",
    "\n",
    "df['Vec_LSA'] = df['content'].apply(lambda x: text2Vec_LSA(x))"
   ]

In [29]:
string = ""
for elem in string_list:
    string+= elem
print(string)

corpus = df["content"].values
dictionary = corpora.Dictionary([d.split() for d in corpus])
corpus_gensim = [dictionary.doc2bow(doc.split()) for doc in corpus]   
tfidf = TfidfModel(corpus_gensim)
corpus_tfidf = tfidf[corpus_gensim]
#lsi = LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)

#On cherche ici le nomre optimal de topic entre les nombres: start et stop
coherence_values = []
model_list = []
start,stop,step=2,10,1
for num_topics in range(start, stop, step):
    # generate LSA model
    print(num_topics)
    model_Lsi = LsiModel(corpus_tfidf, num_topics=num_topics, id2word = dictionary)  # train model
    #model_list.append(model_Lsi) Pour pas refaire le meilleur modèle
    coherencemodel = CoherenceModel(model=model_Lsi, texts=[d.split() for d in corpus], dictionary=dictionary, coherence='c_v')
    coherence_values.append(coherencemodel.get_coherence())

num_topics_optimal = coherence_values.index(max(coherence_values))+ start

print("nombre optimal de topics:"+ num_topi